* Word2Vec
* Dense Neural Network

In [1]:
from src.features.build_features import get_roast_classification_dataset
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
X, y = get_roast_classification_dataset()

In [3]:
X = X.to_numpy()
y = y.to_numpy()

In [4]:
# Split into test and train datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

# validation set
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=25)

In [5]:
VOCAB_SIZE = 2000
max_len = 50
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE, output_mode='int', output_sequence_length=max_len)

In [6]:
encoder.adapt(X_train)

In [7]:
encoder.get_vocabulary()

['',
 '[UNK]',
 'and',
 'in',
 'the',
 'a',
 'cup',
 'finish',
 'aroma',
 'of',
 'with',
 'chocolate',
 'mouthfeel',
 'sweet',
 'acidity',
 'notes',
 'but',
 'rich',
 'flowers',
 'fruit',
 'syrupy',
 'dark',
 'long',
 'cedar',
 'as',
 'to',
 'sweetly',
 'hint',
 'balanced',
 'richly',
 'structure',
 'crisp',
 'milk',
 'is',
 'tart',
 'lightly',
 'cocoa',
 'flavor',
 'hints',
 'floral',
 'into',
 'gently',
 'complex',
 'pungent',
 'by',
 'orange',
 'resonant',
 'nib',
 'delicate',
 'deep',
 'drying',
 'very',
 'round',
 'juicy',
 'bright',
 'full',
 'deeply',
 'zest',
 'silky',
 'coffee',
 'lemon',
 'sweettoned',
 'small',
 'espresso',
 'short',
 'roasted',
 'dry',
 'almond',
 'that',
 'nut',
 'consolidates',
 'honey',
 'crisply',
 'aromatic',
 'cherry',
 'cacao',
 'carry',
 'evaluated',
 'flavorsaturated',
 'citrus',
 'wood',
 'sweetness',
 'body',
 'freshcut',
 'parts',
 'black',
 'rather',
 'slightly',
 'ken',
 'apricot',
 'tones',
 'lively',
 'caramel',
 'an',
 'this',
 'though',
 '

In [8]:
# Integer encode sample sentence
encoder(X)[3200]

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([936,  81,  10, 155,  39,   2,  89,  15,  38,   9,  36,   2,  43,
        99,  48,   1,  14, 113,  16,  51,  58,  12, 374, 418,   5, 190,
         3,   5,  13,  16,  50,   7,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])>

In [9]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [10]:
#encoder(["Bright chocolate structure acidity", "Rich creamy fruity roast"])
#type(X_train[0])
np.array(X_train)

array(['Deep, rich and pungent in aroma and cup. Roundly sweet with flavors that layer dried berries, apricot and spicy tea rose atop dark chocolate. Flavors penetrate deep into a long, resonant finish.',
       'Evaluated as espresso. Richly sweet-tart, wood-framed. Dark chocolate, tangerine zest, fir, hazelnut, a hint of flowers in aroma and small cup. Dense in mouthfeel; bittersweet chocolate dominates in finish. The chocolate reappears in three-parts milk, though its sweetness is limited by a bitterish, wood-toned edge. ',
       'Intense aroma: herb, night flowers, sweet berry. In the cup roast-dampened acidity, leanish mouthfeel but rich flavor: herb, aromatic wood, continued night flowers and a berry note that fattens toward chocolate. The herb and chocolate notes in particular carry into a surprisingly round, smooth finish.',
       ...,
       'Bright and sweetly tart with complex, refreshing temperate fruit notes: cherry, green apple, orange. Smooth, silky mouthfeel. The fini

In [11]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [12]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [13]:
history = model.fit(x=X_train, y=y_train, epochs=10,
                    validation_data=(X_validation, y_validation),
                    validation_steps=30)

Epoch 1/10


NotImplementedError: in user code:

    /Users/brianelinsky/.pyenv/versions/3.9.0/envs/coffee_data/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /Users/brianelinsky/.pyenv/versions/3.9.0/envs/coffee_data/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/brianelinsky/.pyenv/versions/3.9.0/envs/coffee_data/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/brianelinsky/.pyenv/versions/3.9.0/envs/coffee_data/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/brianelinsky/.pyenv/versions/3.9.0/envs/coffee_data/lib/python3.9/sit